# Exploring Reddit Data

The notebook demonstrates how to extract massive amounts of reddit data in order to build a map of subreddits. 

Data extracted from https://files.pushshift.io/reddit/comments/
- Files are around 2-10 times larger when uncompressed
- Uncompressed files must be renamed to have .json file extension

Code adapted from https://github.com/lmcinnes/subreddit_mapping

Credit to reddit user /u/Stuck_In_the_Matrix and /u/hoffa 

In [ ]:
import json
import sqlite3
import sqlalchemy
import adjustText
import subprocess
import numpy as np
import pandas as pd
import scipy.sparse as ss
from os.path import isfile
from sqlalchemy import create_engine
from sklearn.utils import check_array
from sklearn.base import BaseEstimator
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score


import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import HoverTool, ColumnDataSource, value, CustomJS, DataRange1d
from bokeh.models.mappers import LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.palettes import plasma
from bokeh.events import MouseWheel
from collections import OrderedDict
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec
from IPython.display import clear_output

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Build SQL Database

Since the dataset is huge (71,826,552 comments), we need to read and write to the SQL database in chunks. 

In [ ]:
def count_objects_in_file(filepath):
    idx = 0
    with open(filepath) as fp:
        for line in fp:
            idx+=1
    print("File has {} lines".format(idx))


def extract_subset(filepath, start=0, end=10):
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (idx >= start) and (idx < end):
                comment = json.loads(line)
                comments.append(comment)
            elif idx >= end:
                break                
        return comments
        
    
def write_to_database(db_conn, json_fp, chunk_size):
    batch_no=1
    for chunk in pd.read_json(json_fp, chunksize=chunk_size, lines=True):
        try: 
            chunk.to_sql('reddit_comments', db_conn, if_exists='append')
        except sqlalchemy.exc.SQLAlchemyError as e: 
            print("\n  {}".format(e.orig))
        print('\rindex: {}'.format(batch_no), end='')
        batch_no+=1
        
        
def create_database(database, json_fp, n_files, comments_per_file, chunk_size, columns_to_drop):
    for idx in range(0, n_files):
        print("########## File {} ##########".format(idx+1))
        start = int(idx*comments_per_file)
        end = int(start + comments_per_file)
        print("Extracting comments {} - {}".format(start, end))
        comments = extract_subset(filepath=json_fp, start=start, end=end)
        df = pd.DataFrame(comments)
        df = df.drop(columns=columns_to_drop)
        if "media_metadata" in df.columns:
            df = df.drop(columns=["media_metadata"])
        df.to_json("data/db_chunk.json", orient='records', lines=True)

        print("Writing to database")
        write_to_database(
            db_conn=database, 
            json_fp="data/db_chunk.json", 
            chunk_size=chunk_size
        )
        clear_output(wait=True)
        
        
def run_database_builder(input_path, db_path, drop_cols, n_millions):
    
    conn = create_engine('sqlite:///'+db_path)

    create_database(
        database=conn, 
        json_fp=input_path,
        n_files=n_millions, 
        comments_per_file=1000000, 
        chunk_size=100000,
        columns_to_drop=drop_cols
    )

In [ ]:
# count_objects_in_file(filepath="data/RC_2017-10.json")

RC_2017_10 has 85,828,912 comments

#### Attributes to keep
'subreddit_id'
'link_id'
'subreddit'
'created_utc'
'retrieved_on'
'author_flair_text'
'score'
'controversiality'
'author'
'edited'
'id'
'gilded'
'parent_id'
'body'

In [ ]:
# comments = extract_subset(filepath="data/RC_2017-10.json", start=0, end=100)
# df = pd.DataFrame(comments)
# df.columns

In [ ]:
drop_cols = ["stickied", "distinguished", "author_flair_css_class", 
             "can_gild", "is_submitter", "permalink", "author_cakeday"]

In [ ]:
run_database_builder(
    input_path="data/RC_2017-10.json", 
    db_path="databases/RC_2017_10_database.db", 
    drop_cols=drop_cols,
    n_millions=85,
)

### Run SQL queries

In [ ]:
DB_NAME = "databases/RC_2017_10_database.db"
conn = sqlite3.connect(DB_NAME)

In [ ]:
df = pd.read_sql_query("""SELECT * FROM reddit_comments LIMIT 100""", conn)
df = df.drop(columns=["index"])
df.head()

Create a list of number of users in each subreddit, and save the result a new table

In [ ]:
QUERY1 = """
CREATE TABLE subr_users AS
    SELECT subreddit, authors, DENSE_RANK() OVER (ORDER BY authors DESC) AS rank_authors
    FROM (SELECT subreddit, SUM(1) as authors
         FROM (SELECT subreddit, author, COUNT(1) as cnt 
             FROM reddit_comments
             WHERE author NOT LIKE '%bot'
             GROUP BY subreddit, author HAVING cnt > 0)
         GROUP BY subreddit) t
    ORDER BY authors DESC;
"""

c = conn.cursor()
c.execute(QUERY1)
conn.commit()

In [ ]:
# c = conn.cursor()
# c.execute("""DROP TABLE subr_users_v2""")
# conn.commit()

In [ ]:
df = pd.read_sql_query("""SELECT * FROM subr_users""", conn)
df.head()

In [ ]:
plt.figure(figsize=(16,6))

plt.subplot(121)
plt.title("authors")
plt.plot(df.index, df.authors)
plt.yscale("log")

plt.subplot(122)
plt.title("authors hist")
plt.hist(df.authors, bins=50)
plt.yscale("log")

plt.show()

In [ ]:
plt.figure(figsize=(16,6))

plt.title("authors")
plt.plot(df.index[200:2201], df.authors[200:2201])
plt.show()

Using the new table, we create a list of number of users who authored at least 10 posts in pairs of subreddits

In [ ]:
QUERY2 = """
CREATE TABLE overlapping_subr_users AS
    SELECT t1.subreddit, t2.subreddit, SUM(1) AS NumOverlaps
    FROM (SELECT subreddit, author, COUNT(1) AS cnt 
         FROM reddit_comments
         WHERE author NOT LIKE '%bot'
         AND subreddit IN (SELECT subreddit FROM subr_users
           WHERE rank_authors>200 AND rank_authors<2201)
         GROUP BY subreddit, author HAVING cnt > 10) t1
    JOIN (SELECT subreddit, author, COUNT(1) as cnt 
         FROM reddit_comments
         WHERE author NOT LIKE '%bot'
         GROUP BY subreddit, author HAVING cnt > 10) t2
    ON t1.author=t2.author
    WHERE t1.subreddit!=t2.subreddit
    GROUP BY t1.subreddit, t2.subreddit
"""

c = conn.cursor()
c.execute(QUERY2)
conn.commit()

In [ ]:
# c.execute("""DROP TABLE overlapping_subr_users_v2""")
# conn.commit()

Read in the result of the second query as a dataframe. Edit the column names and store it for later

In [ ]:
df = pd.read_sql_query("""SELECT * FROM overlapping_subr_users""", conn)
df = df.rename(columns={"subreddit":"t1_subreddit", "subreddit:1":"t2_subreddit"})
df.describe()

In [ ]:
df.head()

In [ ]:
print("t1_subreddit unique subreddits: {}".format(len(df["t1_subreddit"].unique())))
print("t2_subreddit unique subreddits: {}".format(len(df["t2_subreddit"].unique())))

In [ ]:
df.to_csv("data/RC_2017_10_subreddit_overlaps.csv", index=False)

Close the database connection

In [ ]:
conn.close()

# Subreddit Mapping

In [ ]:
raw_data = pd.read_csv("data/RC_2017_10_subreddit_overlaps.csv")
# raw_data = pd.read_csv("data/subreddit_overlaps_BQ.csv")

In [ ]:
raw_data

In [ ]:
raw_data.info()

In [ ]:
raw_data.describe()

In [ ]:
print("Number of pairwise commenter overlaps: {}".format(len(raw_data)))
print("t1_subreddit unique subreddits: {}".format(len(raw_data["t1_subreddit"].unique())))
print("t2_subreddit unique subreddits: {}".format(len(raw_data["t2_subreddit"].unique())))

Rank the subreddits so that they are indexed in order of popularity. Popularity is defined by the total number of unique commenters in each subreddit. 

In [ ]:
subreddit_popularity = raw_data.groupby('t2_subreddit')['NumOverlaps'].sum()
subreddits = np.array(subreddit_popularity.sort_values(ascending=False).index)

In [ ]:
subreddits.tolist()[0:5]

Pivot the data into a matrix such that rows and columns are both indexed by subreddits, and the entry at position (i,j) is the number of overlaps bwteen the ith and jth subreddits

Create subreddit-to-integer-index map to convert the subreddit names in the table into numeric row and column indexes.

In [ ]:
index_map = dict(np.vstack([subreddits, np.arange(subreddits.shape[0])]).T)

In [ ]:
values = raw_data.NumOverlaps
row_indices = raw_data.t2_subreddit.map(index_map)
col_indices = raw_data.t1_subreddit.map(index_map)

Create a sparse matrix. This format requires us to specify triples of row, column, and value for each non-zero entry in the matrix. The COO matrix constructor accepts this as a triple of arrays: the first array is the values, the second and third are arrays of row and column indices.

In [ ]:
count_matrix = ss.coo_matrix((values, (row_indices,col_indices)),
                              shape=(subreddits.shape[0], subreddits.shape[0]),
                              dtype=np.float64)

In [ ]:
count_matrix.shape

In [ ]:
conditional_prob_matrix = count_matrix.tocsr()
conditional_prob_matrix = normalize(conditional_prob_matrix, norm='l1', copy=False)

## Converting subreddit vectors into a map

### Linear dimensionality reduction down to 500 dimensions

In [ ]:
reduced_vectors = TruncatedSVD(n_components=500, random_state=1).fit_transform(conditional_prob_matrix)
reduced_vectors = normalize(reduced_vectors, norm='l2', copy=False)

In [ ]:
reduced_vectors.shape

### Nonlinear dimensionality reduction down to 2 dimensions

In [ ]:
class LargeVis(BaseEstimator):
    
    def __init__(self, n_components=2, perplexity=30.0, gamma=5,
                 layout_samples=None, n_neighbors=None, negative_samples=5,
                 alpha=1.0, n_cores=6, knn_prop=3, trees=50):
        self.n_components = n_components
        self.perplexity = perplexity
        self.layout_samples = layout_samples
        self.alpha = alpha
        self.n_cores = n_cores
        self.knn_prop = knn_prop
        self.negative_samples = negative_samples
        self.n_neighbors = n_neighbors
        self.gamma = gamma
        self.trees = trees
        if self.n_neighbors is None:
            self.n_neighbors = int(self.perplexity * 3)


    def fit_transform(self, X, y=None):
        
        if self.layout_samples is None:
            layout_samples = X.shape[0] / 100.0
        else:
            layout_samples = self.layout_samples
            
        X = check_array(X, dtype=np.float64)
        np.savetxt('/tmp/largevis_input', 
                   X, header='{} {}'.format(*X.shape), 
                   comments='')
        subprocess.check_call(['/Users/cameronlaedtke/LargeVis-python3/Linux/LargeVis',
                               '-input', '/tmp/largevis_input',
                               '-output', '/tmp/largevis_output',
                               '-outdim', str(self.n_components),
                               '-perp', str(self.perplexity),
                               '-samples', str(layout_samples),
                               '-gamma', str(self.gamma),
                               '-prop', str(self.knn_prop),
                               '-trees', str(self.trees),
                               '-neigh', str(self.n_neighbors),
                               '-alpha', str(self.alpha),
                               '-neg', str(self.negative_samples),
                               '-threads', str(self.n_cores)])
        self.embedding_ = np.loadtxt('/tmp/largevis_output', skiprows=1)
        return self.embedding_
    
    
    def fit(self, X, y=None):
        self.fit_transform(X)
        return self

In [ ]:
embed_file = 'tsne_data/RC_2017_10_largevis_subreddit_map.npy'
if isfile(embed_file):
    subreddit_map = np.load(embed_file)
else:
    largevis = LargeVis(perplexity=30, n_cores=12)
    subreddit_map = largevis.fit_transform(reduced_vectors[:10000])
    np.save(embed_file, subreddit_map)

In [ ]:
# largevis = LargeVis(perplexity=30, n_cores=12)
# subreddit_map = largevis.fit_transform(reduced_vectors[:10000])
# np.save(embed_file, subreddit_map)

In [ ]:
subreddit_map_df = pd.DataFrame(subreddit_map, columns=('x', 'y'))
subreddit_map_df['subreddit'] = subreddits[:10000]
subreddit_map_df.head()

### Clustering the map

In [ ]:
clusterer = HDBSCAN(min_samples=5, min_cluster_size=20).fit(subreddit_map)
cluster_ids = clusterer.labels_

In [ ]:
subreddit_map_df['cluster'] = cluster_ids
subreddit_map_df.head()

In [ ]:
n_cluster_points = len(subreddit_map_df[subreddit_map_df.cluster != -1])
n_clusters = subreddit_map_df["cluster"].max()
score = silhouette_score(subreddit_map, cluster_ids)
print("points assigned to a cluster: {}".format(n_cluster_points))
print("number of clusters: {}".format(n_clusters))
print("silhouette score: {:.4f}".format(score))

### Visualization

In [ ]:
# output_notebook()

In [ ]:
# Construct a color palette and map clusters to colors
palette = ['#777777'] + list(plasma(cluster_ids.max()))
colormap = LinearColorMapper(palette=palette, low=-1, high=cluster_ids.max())
color_dict = {'field': 'cluster', 'transform': colormap}

# Set fill alpha globally
subreddit_map_df['fill_alpha'] = np.exp((subreddit_map.min() - subreddit_map.max()) / 5.0) + 0.05

# Build a column data source
plot_data = ColumnDataSource(data=subreddit_map_df)

# Create the figure and add tools
fig = figure(
    title='A Map of Subreddits',
    plot_width = 1000,
    plot_height = 1000,
    tools= ('pan, wheel_zoom, box_zoom,''box_select, reset'),
    active_scroll=u'wheel_zoom',
)

fig.add_tools(HoverTool(tooltips = OrderedDict([('subreddit', '@subreddit'), 
                                                ('cluster', '@cluster')])))

# draw the subreddits as circles on the plot
fig.circle(
    u'x', u'y', 
    source = plot_data,
    fill_color = color_dict, 
    line_color = None, 
    fill_alpha = 'fill_alpha',
    size = 10, 
    hover_line_color = u'black'
)

# Custom callback for alpha adjustment
jscode="""
    var data = source.data;
    var start = cb_obj.start;
    var end = cb_obj.end;
    var alpha = data['fill_alpha']
    for (var i = 0; i < alpha.length; i++) {
         alpha[i] = Math.exp((start - end) / 5.0) + 0.05;
    }
    source.change.emit();
"""

callback = CustomJS(args=dict(source=plot_data), code=jscode)

fig.x_range.js_on_change("start", callback)
fig.x_range.js_on_change("end", callback)

# configure visual elements of the plot
fig.title.text_font_size = value('18pt')
fig.title.align = 'center'
fig.xaxis.visible = False
fig.yaxis.visible = False
fig.grid.grid_line_color = None
fig.outline_line_color = '#222222'

# display the figure
output_file('viz/subreddit_interactive_map.html')
show(fig);